# Setup

A helper function to define the naming conventions and create ini file commands for BBS

In [16]:
import pandas as pd


Eco = ['PSW','PSTS','PSLS']
Treatment = ['S','R']
Depth = ['5cm','10cm','30cm']
# List = []
Names = {
    'Clean_Name':[],
    'Met_Name':[],
    'Units':[],
    'minMax':[]
}
h = 1
for e in Eco:
    for i,t in enumerate(Treatment):
        for v,d in enumerate(Depth):
            Names['Clean_Name'].append(f'TS_{h}_{v+1}_1')
            Names['Met_Name'].append(f'{e}_{t}_TSoil_{d}')
            Names['Units'].append('C')
            Names['minMax'].append('[-5,35]')
        h += 1
        
met = ['PSW_S_VWC','PSW_S_WTD','PSW_R_VWC']
clean = ['VWC_1_1_1','VWC_2_1_1','WTD_1_1_1']
units = ['%','%','m']
minMax = ['[0,1]','[0,1]','[0,5]']
for met,clean,u,m in zip(met,clean,units,minMax):
    Names['Met_Name'].append(met)
    Names['Clean_Name'].append(clean)
    Names['Units'].append(u)
    Names['minMax'].append(m)


df = pd.DataFrame(data=Names)#,index=['Clean_Name','Type']).T
# df.index.name = 'Met_Name'
df

,Clean_Name,Met_Name,Units,minMax
0,TS_1_1_1,PSW_S_TSoil_5cm,C,"[-5,35]"
1,TS_1_2_1,PSW_S_TSoil_10cm,C,"[-5,35]"
2,TS_1_3_1,PSW_S_TSoil_30cm,C,"[-5,35]"
3,TS_2_1_1,PSW_R_TSoil_5cm,C,"[-5,35]"
4,TS_2_2_1,PSW_R_TSoil_10cm,C,"[-5,35]"
5,TS_2_3_1,PSW_R_TSoil_30cm,C,"[-5,35]"
6,TS_3_1_1,PSTS_S_TSoil_5cm,C,"[-5,35]"
7,TS_3_2_1,PSTS_S_TSoil_10cm,C,"[-5,35]"
8,TS_3_3_1,PSTS_S_TSoil_30cm,C,"[-5,35]"
9,TS_4_1_1,PSTS_R_TSoil_5cm,C,"[-5,35]"


## Stage 1

In [17]:
Template = '''

[Trace]
	variableName = 'Clean_Name'
	title = 'See Soil_ini_Helper.ipynb'
	originalVariable = 'See Soil_ini_Helper.ipynb'
	inputFileName = {'Met_Name'}
	inputFileName_dates =[datenum(2023,1,1) datenum(2100,1,1)]
	measurementType = 'Met'
	units = 'Units' 
	instrument = '-'
	instrumentSN = ''
	calibrationDates = '' 
	loggedCalibration = ''
	currentCalibration = ''
	comments = ''
	minMax = [minMax]
	clamped_minMax = [minMax]
	zeroPt = [-9999]
	plotBottomRight = ''
	plotBottomLeft  = ''
	dependent       = ''
[End]

'''

Stage_1 = '%%%% Data from CR10_Stations'

for i,row in df.iterrows():
    Stage_1 += Template.replace('Clean_Name',row['Clean_Name']).replace('Met_Name',row['Met_Name']).replace('Units',row['Units']).replace('[minMax]',row['minMax'])
    
with open('BBS_FirstStage_Template.ini') as file:
    file = file.read()
    Stage1_Out = file.replace('%%%% Data from CR10_Stations',Stage_1)
    
with open('BBS_FirstStage.ini','+w') as file:
    file.write(Stage1_Out)

## Stage 2

In [18]:
Template = '''

[Trace]
	variableName = 'Clean_Name'
	Evaluate = 'Clean_Name = Clean_Name;'
	title = '-'
	units = 'Units'
	minMax = [minMax]
[End]

'''

Stage_2 = '%%%% Data from CR10_Stations'

for i,row in df.iterrows():
    Stage_2 += Template.replace('Clean_Name',row['Clean_Name']).replace('Met_Name',row['Met_Name']).replace('Units',row['Units']).replace('[minMax]',row['minMax'])
    
with open('BBS_SecondStage_Template.ini') as file:
    file = file.read()
    Stage2_Out = file.replace('%%%% Data from CR10_Stations',Stage_2)
    
with open('BBS_SecondStage.ini','+w') as file:
    file.write(Stage2_Out)